# Dense model

![dense_model.png](attachment:dense_model.png width="300")
<img src="attachment:image.png" width="400">

In [1]:
import tensorflow as tf
from util import dataset_fn
from dense_model import Model

In [2]:
TRAIN_CORPUS_PATH = './data/ratings_train.cleaned.txt'
TEST_CORPUS_PATH = './data/ratings_test.cleaned.txt'
VOCAB_PATH = 'Word2vec.vocab'
VOCAB_MODEL_PATH = 'Word2vec.model'
MODEL_PATH = 'Dense.model'

MAX_LEN = 100
BATCH_SIZE = 128
LEARNING_RATE = 0.001
TRAIN_STEPS = 10000

In [3]:
train_dataset = dataset_fn(TRAIN_CORPUS_PATH, VOCAB_PATH, MAX_LEN, BATCH_SIZE)
test_dataset = dataset_fn(TEST_CORPUS_PATH, VOCAB_PATH, MAX_LEN, BATCH_SIZE)

iters = tf.data.Iterator.from_structure(train_dataset.output_types,
                                        train_dataset.output_shapes)
features = iters.get_next()

# create the initialisation operations
train_init_op = iters.make_initializer(train_dataset)
test_init_op = iters.make_initializer(test_dataset)

print("Now building model")
model = Model(VOCAB_MODEL_PATH, MAX_LEN, LEARNING_RATE)
loss, acc, opt = model.build_optimizer(features)

Now building model


In [4]:
## for tensorboard
train_loss_graph = tf.placeholder(shape=None, dtype=tf.float32)
train_acc_graph = tf.placeholder(shape=None, dtype=tf.float32)
test_loss_graph = tf.placeholder(shape=None, dtype=tf.float32)
test_acc_graph = tf.placeholder(shape=None, dtype=tf.float32)

In [5]:
print("Now training")

saver = tf.train.Saver()
ckpt = tf.train.get_checkpoint_state("./model")

summary_train_loss = tf.summary.scalar("train_loss", train_loss_graph)
summary_train_acc = tf.summary.scalar("train_acc", train_acc_graph)
summary_test_loss = tf.summary.scalar("test_loss", test_loss_graph)
summary_test_acc = tf.summary.scalar("test_acc", test_acc_graph)
merged_train = tf.summary.merge([summary_train_loss, summary_train_acc])
merged_test = tf.summary.merge([summary_test_loss, summary_test_acc])

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    sess.run(train_init_op)
    sess.run(tf.tables_initializer())

    n_train_step = 0
    train_loss_, train_acc_ = 0., 0.
    test_loss_, test_acc_ = 0., 0.
    best_loss = 1e8
    writer = tf.summary.FileWriter('./tensorboard/graph', sess.graph)

    for step in range(TRAIN_STEPS):
        n_train_step += 1
        batch_train_loss, batch_train_acc, _ = sess.run([loss, acc, opt])
        train_loss_ += batch_train_loss
        train_acc_ += batch_train_acc
        train_loss = train_loss_ / n_train_step
        train_acc = train_acc_ / n_train_step
        
        if step % 10 == 0 and step > 0:
            print(f"step: {step + 1} train_loss: {train_loss} train_acc: {train_acc}")

        if step % 100 == 0 and step > 0:
            summary = sess.run(merged_train,
                               feed_dict={train_loss_graph: train_loss, train_acc_graph: train_acc})
            writer.add_summary(summary, step)

        if step % 2000 == 0 and step > 0:
            print("Now for test data")
            sess.run(test_init_op)
            n_test_step = 0

            try:
                while True:
                    n_test_step += 1
                    batch_test_loss, batch_test_acc = sess.run([loss, acc])
                    test_loss_ += batch_test_loss
                    test_acc_ += batch_test_acc

            except tf.errors.OutOfRangeError:
                pass

            test_loss = test_loss_ / n_test_step
            test_acc = test_acc_ / n_test_step

            summary = sess.run(merged_test, feed_dict={test_loss_graph: test_loss, test_acc_graph: test_loss})
            writer.add_summary(summary, step)
            print(f"step: {step + 1} test_loss: {test_loss} test_acc: {test_acc}")

            if test_loss < best_loss or step % 2000 == 0:
                save_path = saver.save(sess, "./model/" + MODEL_PATH)
                best_loss = test_loss

            sess.run(train_init_op)

Now training
step: 11 train_loss: 0.6033951504663988 train_acc: 0.6626420454545454
step: 21 train_loss: 0.5394948763506753 train_acc: 0.7276785714285714
step: 31 train_loss: 0.5057595747132455 train_acc: 0.7550403225806451
step: 41 train_loss: 0.4961309709200045 train_acc: 0.7682926829268293
step: 51 train_loss: 0.48953339106896343 train_acc: 0.7766544117647058
step: 61 train_loss: 0.47748040908672773 train_acc: 0.7820184426229508
step: 71 train_loss: 0.47311739224783134 train_acc: 0.7869718309859155
step: 81 train_loss: 0.4672812418437298 train_acc: 0.7910879629629629
step: 91 train_loss: 0.46103066500726636 train_acc: 0.7958447802197802
step: 101 train_loss: 0.4561078223851648 train_acc: 0.7991955445544554
step: 111 train_loss: 0.4509219113770906 train_acc: 0.8020129504504504
step: 121 train_loss: 0.44989592625089914 train_acc: 0.8032670454545454
step: 131 train_loss: 0.44772606496592515 train_acc: 0.8048664122137404
step: 141 train_loss: 0.4459563897433856 train_acc: 0.8064051418439

step: 1161 train_loss: 0.4227524366130303 train_acc: 0.8196934216192937
step: 1171 train_loss: 0.4228201711361068 train_acc: 0.8196319918872759
step: 1181 train_loss: 0.42286133690469857 train_acc: 0.8197039055884844
step: 1191 train_loss: 0.4230417038941964 train_acc: 0.8196762174643157
step: 1201 train_loss: 0.4229446557985555 train_acc: 0.8197270503746877
step: 1211 train_loss: 0.42290729257215853 train_acc: 0.8198028488852188
step: 1221 train_loss: 0.4228308087338394 train_acc: 0.8198646089271089
step: 1231 train_loss: 0.42279765247136575 train_acc: 0.8199634443541836
step: 1241 train_loss: 0.42263848941524024 train_acc: 0.8200103243352136
step: 1251 train_loss: 0.4225582443267036 train_acc: 0.8201189048760992
step: 1261 train_loss: 0.42223254798613874 train_acc: 0.8202815226011102
step: 1271 train_loss: 0.4220616096684916 train_acc: 0.8203063532651456
step: 1281 train_loss: 0.42213183988080555 train_acc: 0.8203490925058547
step: 1291 train_loss: 0.4219343454268993 train_acc: 0.820

KeyboardInterrupt: 

## Tensorboard

- tensorboard --logdir=./tensorboard/graph/